In [2]:
import pyrosetta
pyrosetta.init()

from multiprocessing import Pool
import logging

import itertools
import matplotlib
import pandas as pd
import seaborn

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: 2018.51.post.dev+654.v2019.01dev60566.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-1240895222 seed_offset=0 real_seed=-1240895222
core.init.random: RandomGenerator:init: Normal mode, seed=-1240895222 RG_type=mt19937


In [3]:
# import pose from pdb

pdb = '1stn-relaxed.pdb'
relaxed_pose = pyrosetta.pose_from_pdb(pdb)

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 636 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.923091 seconds.
core.import_pose.import_pose: File '1stn-relaxed.pdb' automatically determined to be of type PDB


In [5]:
# make backrub ensemble from WT

n_backrub = 2

backrub_protocol = pyrosetta.rosetta.protocols.backrub.BackrubProtocol()
backrub_pose_list = []

for i in range(n_backrub):
    pose = relaxed_pose.clone()
    pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, "structure_num", i)
    backrub_protocol.apply(pose)
    backrub_pose_list.append(pose)

core.mm.MMBondAngleLibrary: MM bond angle sets added fully assigned: 603; wildcard: 0 and 1 virtual parameter.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.365195 seconds to load from binary
protocols.evaluation.ChiWellRmsdEvaluatorCreator: Evaluation Creator active ...
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.e

protocols.moves.TrialCounter:      br_CA_CA_34 trials=     68;  accepts= 0.1912;  energy_drop/trial=  -0.17216
protocols.moves.TrialCounter:       sc_chi_rot trials=    195;  accepts= 0.1487;  energy_drop/trial=  -0.03625
protocols.moves.TrialCounter:      sc_chi_unif trials=     18 NO ACCEPTS.
protocols.backrub.BackrubProtocol: Last Score:
protocols.backrub.BackrubProtocol: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -752.466    -752.466
 fa_rep                       0.550     185.804     102.192
 fa_sol                       1.000     490.963     490.963
 fa_intra_rep                 0.005     303.280       1.516
 fa_intra_sol_xover4          1.000      27.859      27.859
 lk_ball_wtd                  1.000     -17.753     -17.753
 mm_bend                      1.000      72.683      72.683
 fa_elec          

protocols.moves.TrialCounter:      br_CA_CA_25 trials=     58;  accepts= 0.2241;  energy_drop/trial=   0.00820
protocols.moves.TrialCounter:      br_CA_CA_28 trials=     76;  accepts= 0.1974;  energy_drop/trial=  -0.06172
protocols.moves.TrialCounter:      br_CA_CA_31 trials=     67;  accepts= 0.1045;  energy_drop/trial=  -0.02304
protocols.moves.TrialCounter:      br_CA_CA_34 trials=     68;  accepts= 0.1912;  energy_drop/trial=  -0.02557
protocols.moves.TrialCounter:       sc_chi_rot trials=    213;  accepts= 0.2019;  energy_drop/trial=  -0.03517
protocols.moves.TrialCounter:      sc_chi_unif trials=     24 NO ACCEPTS.
protocols.backrub.BackrubProtocol: Last Score:
protocols.backrub.BackrubProtocol: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -750.354    -750.354
 fa_rep                       0.550     184.0

In [21]:
# read in mutations from Kellogg dataset

df = pd.read_csv('./kellogg.csv', comment = '#')
df['chain'], df['wt_res'], df['pdb_res_index'], df['mut_res'] = df['Mutations'].str.split(' ', 3).str
df.pdb_res_index = df.pdb_res_index.astype(int)
df = df.drop('Mutations', axis=1)
df.head()

,RecordID,PDBFileID,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,chain,wt_res,pdb_res_index,mut_res
0,1,107L,-0.53,0.435294117647,H,H,0.53,A,G,44,S
1,2,160L,-0.20,0.070796460177,H,H,0.2,A,A,120,M
2,3,161L,0.17,0.212389380531,H,H,-0.17,A,A,116,N
3,4,162L,-0.24,0.389380530973,H,H,0.24,A,A,122,Q
4,5,163L,-0.22,0.46017699115,T,O,0.22,A,A,123,Q


In [23]:
# pick mutations based on pdb

pdb_id = pdb[:4].upper()
pdb_ddgs = df[df.PDBFileID == pdb_id]
pdb_ddgs['res_index'] = pdb_ddgs.apply(lambda x: relaxed_pose.pdb_info().pdb2pose(x.chain, int(x.pdb_res_index)), axis=1)

print('PDB: {}'.format(pdb_id))
print('Max ddG = {}'.format(pdb_ddgs.DDG.max()))
print('Min ddG = {}'.format(pdb_ddgs.DDG.min()))

pdb_ddgs.head()

PDB: 1STN
Max ddG = 7.5
Min ddG = -1.25


/Users/cjmathy/miniconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,RecordID,PDBFileID,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,chain,wt_res,pdb_res_index,mut_res,res_index
581,706,1STN,5.2,0.0,H,H,5.2,A,N,100,A,95
582,707,1STN,5.1,0.0,H,H,5.1,A,N,100,G,95
583,558,1STN,1.9,0.229508196721,H,H,1.9,A,E,101,A,96
584,559,1STN,3.1,0.229508196721,H,H,3.1,A,E,101,G,96
585,515,1STN,1.9,0.150442477876,H,H,1.9,A,A,102,F,97


In [24]:
# prepare mutation arguments from dataframe
mutations = list(zip(pdb_ddgs.res_index, pdb_ddgs.mut_res))

# setup tuples for multiprocessing
work = [(input_pose, res_index, new_aa)
        for input_pose, (res_index, new_aa)
        in itertools.product(backrub_pose_list, mutations)] 

In [25]:
def mutate_minimize(input_pose, res_index, new_aa):
    '''Adapted from an example by Alex Ford and Brian Weitzner at Pre-RosettaCON 2018'''
    
    #work_pose = packed_pose.to_pose(input_pose)
    work_pose = input_pose
    
    # Add parameters as score, for use in downstream analysis
    pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_index", res_index)
    pyrosetta.rosetta.core.pose.setPoseExtraScore(work_pose, "mutation_aa", new_aa)
    
    #Mutate residue
    pyrosetta.toolbox.mutants.mutate_residue(work_pose, res_index, new_aa)
    
    #Minimize inner shell and whole protein
    minimizer(work_pose, res_index)
    
    return work_pose

In [26]:
def minimizer(work_pose, res_index):
    
    #Score function
    sfxn = pyrosetta.create_score_function("ref2015_cart.wts")
    
    #Get neighborhood of residue being mutated, 6 A 
    close_residues = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(res_index, 6.0, True)
    far_residues = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(close_residues)
    
    #MovemapFactory
    mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
    mmf.all_bb(setting=True)  # Set to true if needed
    mmf.all_bondangles(setting=True)
    mmf.all_bondlengths(setting=True)
    mmf.all_chi(setting=True)
    mmf.set_cartesian(setting=True)
    
    # Set movemap actions to turn on or off certain areas, overriding the above defaults
    #enable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_enable
    #disable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_disable
    #mmf.add_bondangles_action(action=disable, selector=notselected)
    #mmf.add_bondlengths_action(action=disable, selector=notselected)
    #mmf.add_chi_action(action=disable, selector=chain_BC_not_interface)
    
    #Foldtree: to make mutation affect in both directions include bidirectional movement in FT
    # This will be useful in FastRelax
    ft = pyrosetta.rosetta.core.kinematics.FoldTree()
    ft.add_edge(1, res_index, -1)
    ft.add_edge(res_index, work_pose.total_residue(), -1)
    work_pose.fold_tree(ft)
    
    #Make Task Factory
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

    inner_shell = operation.OperateOnResidueSubset(close_residues)
    outer_shell = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(close_residues)
    
    tf.push_back(inner_shell)
    tf.push_back(outer_shell)
    
    #Minmover
    min_mover = pyrosetta.rosetta.protocols.minimization_packing.MinMover() 
    min_mover.set_movemap(mmf) 
    min_mover.score_function(sfxn)
    min_mover.apply(work_pose)
    
    #Pack the area 6 A away from mutated residue only
    packer = pack_min.PackRotamersMover()
    packer.task_factory(tf) 
    prevent_repacking_rlt = operation.PreventRepackingRLT()
    prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, outer_shell, True )
    tf.push_back(prevent_subset_repacking)
    packer.task_factory(tf)
    packer.apply(pose)
    

In [ ]:
# run mutate and minimize, using multiprocessing
with pyrosetta.distributed.utility.log.LoggingContext(logging.getLogger("rosetta"), level=logging.WARN):
    with Pool() as p:         
        logging.info("Now generating mutations and minimizing")
        all_structures = p.starmap(mutate_minimize, work)